In [ ]:
pip install pandas numpy matplotlib seaborn scikit-learn xgboost


In [ ]:
import os
print(os.getcwd())


/content


In [ ]:
from google.colab import files
uploaded = files.upload()


In [ ]:
import pandas as pd
data = pd.read_csv('diabetes dataset.csv')   # put the downloaded file in your working folder
data.head()
data.info()
data.describe()


FileNotFoundError: [Errno 2] No such file or directory: 'diabetes dataset.csv'

In [ ]:
#exploratory data analysis
import seaborn as sns
import matplotlib.pyplot as plt

# Class balance
print(data['Outcome'].value_counts(normalize=True))

# Histograms
data.hist(figsize=(12,10))
plt.tight_layout()

# Correlation heatmap
plt.figure(figsize=(8,6))
sns.heatmap(data.corr(), annot=True, fmt='.2f', cmap='coolwarm')
plt.show()


In [ ]:
#Data preprocessing — missing values & invalid zeros
import numpy as np

cols_with_zero = ['Glucose','BloodPressure','SkinThickness','Insulin','BMI']
for col in cols_with_zero:
    data[col] = data[col].replace(0, np.nan)

# Show missing counts
print(data.isnull().sum())


In [ ]:
#Imputation options: mean/median or KNN imputer. For simplicity use median (robust to outliers).
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='median')
data[cols_with_zero] = imputer.fit_transform(data[cols_with_zero])


In [ ]:
#Train/test split
from sklearn.model_selection import train_test_split

X = data.drop('Outcome', axis=1)
y = data['Outcome']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.20, stratify=y, random_state=42)


In [ ]:
#Baseline model — Logistic Regression (with pipeline)
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression

pipe_lr = Pipeline([
    ('scaler', StandardScaler()),
    ('clf', LogisticRegression(max_iter=1000, random_state=42))
])

pipe_lr.fit(X_train, y_train)
y_pred = pipe_lr.predict(X_test)
y_prob = pipe_lr.predict_proba(X_test)[:,1]


In [ ]:
#Evaluation metrics (accuracy, precision, recall, f1, ROC-AUC)
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix, classification_report, roc_curve

print("Accuracy:", accuracy_score(y_test, y_pred))
print("Precision:", precision_score(y_test, y_pred))
print("Recall:", recall_score(y_test, y_pred))
print("F1:", f1_score(y_test, y_pred))
print("ROC-AUC:", roc_auc_score(y_test, y_prob))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

# Plot ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
plt.figure()
plt.plot(fpr, tpr, label=f"AUC = {roc_auc_score(y_test, y_prob):.3f}")
plt.plot([0,1],[0,1],'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend()
plt.show()


In [ ]:
#Try SVM (alternative model)
from sklearn.svm import SVC
pipe_svc = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', SVC(probability=True, random_state=42))
])
pipe_svc.fit(X_train, y_train)
y_pred_svc = pipe_svc.predict(X_test)
y_prob_svc = pipe_svc.predict_proba(X_test)[:,1]

print("SVM ROC-AUC:", roc_auc_score(y_test, y_prob_svc))


In [ ]:
#Cross-validation & model comparison
from sklearn.model_selection import cross_val_score, StratifiedKFold

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
for name, model in [('Logistic Regression', pipe_lr), ('SVM', pipe_svc)]:
    scores = cross_val_score(model, X, y, scoring='roc_auc', cv=cv)
    print(name, "ROC-AUC mean ± std:", scores.mean(), scores.std())


In [ ]:
#Hyperparameter tuning (GridSearchCV)
from sklearn.model_selection import GridSearchCV

param_grid = {
    'clf__C': [0.01, 0.1, 1, 10, 100],
    'clf__penalty': ['l2'],
    'clf__solver': ['lbfgs']
}
gs = GridSearchCV(pipe_lr, param_grid, scoring='roc_auc', cv=cv, n_jobs=-1)
gs.fit(X_train, y_train)
print("Best params:", gs.best_params_)
print("Best CV ROC-AUC:", gs.best_score_)


In [ ]:
#Feature selection / importance
#Logistic regression coefficients:
coef = gs.best_estimator_.named_steps['clf'].coef_[0]  # if you used gs
feature_importance = pd.Series(abs(coef), index=X.columns).sort_values(ascending=False)
print(feature_importance)
feature_importance.plot(kind='bar')
plt.title("Feature importance (Logistic Regression coefficients)")
plt.show()


In [ ]:
#Random Forest importances:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=42)
rf.fit(X_train, y_train)
pd.Series(rf.feature_importances_, index=X.columns).sort_values(ascending=False).plot(kind='bar')
plt.title("Feature importance (Random Forest)")
plt.show()


In [ ]:
#RFE example:
from sklearn.feature_selection import RFE
selector = RFE(estimator=LogisticRegression(max_iter=1000), n_features_to_select=5)
selector.fit(X_train, y_train)
selected_features = X.columns[selector.support_]
print("Top features by RFE:", list(selected_features))


In [ ]:
#Dealing with class imbalance (if present)
LogisticRegression(class_weight='balanced')


In [ ]:
#Final evaluation & save model
import joblib
best_model = gs.best_estimator_  # or any final pipeline
best_model.fit(X_train, y_train)
joblib.dump(best_model, 'diabetes_model.pkl')


In [ ]:
import joblib

# assuming your best model is stored in variable 'best_model'
joblib.dump(best_model, 'diabetes_model.pkl')


In [ ]:
from google.colab import files
files.download('diabetes_model.pkl')
